# Εργασία στην Αναγνώρηση Προτύπων και Μηχανική Μάθηση
## Μέρος Δ
## ΕΠΙΜΕΛΕΙΑ :
* Ομάδα 25
* Γουρδομιχάλης Αναστάσιος 10333
* Πίττης Γεώργιος 10586

---

# ΜΕΡΟΣ Δ

**Η λογική που ακολουθήσαμε στο θέμα Δ είναι η εξής:**

### Χωρισμός του συνολικού training set:
   - Χωρίσαμε το συνολικό training set των 8743 δειγμάτων σε 2 υποσύνολα:
     - **Σύνολο εκπαίδευσης** [ X_train, y_train ] που είναι το 80% του συνολικού training set.
     - **Validation set** [ X_val, y_val ] που ταυτίζεται με το 20% του συνολικού training set.
   - Τα **X_train**, **X_val** περιέχουν τα 224 χαρακτηριστικά (features) των δειγμάτων.
   - Τα **y_train**, **y_val** περιέχουν τις ετικέτες (labels) των δειγμάτων.

### Manual hyperparameter tuning για κάθε αλγόριθμο ταξινόμησης:
   - Εφαρμόσαμε manual hyperparameter tuning για τους παρακάτω αλγορίθμους ταξινόμησης:
     - Random Forest (Τυχαίο Δάσος)
     - SVM (Support Vector Machine)
     - Gaussian Naive Bayes
     - MLP (Multilayer Perceptron)
     - k-Nearest Neighbors (KNN)
     - Decision Tree (Δέντρο Απόφασης)
     - Multinomial Logistic Regression (Λογιστική Παλινδρόμηση για Πολλαπλές Κλάσεις)
   - Στη διαδικασία του tuning, εκπαιδεύσαμε διάφορα μοντέλα από κάθε αλγόριθμο ταξινόμησης στο [ X_train, y_train ] και τα τεστάραμε στο X_val.
   - Με βάση την ακρίβεια (accuracy) στο validation set, επιλέξαμε το καλύτερο μοντέλο για κάθε αλγόριθμο.

### Επιλογή του συνολικού καλύτερου μοντέλου:
   - Αφού ολοκληρώσαμε το tuning για όλους τους αλγορίθμους, επιλέξαμε το συνολικό καλύτερο μοντέλο.

### Βελτίωση απόδοσης με PCA (Principal Component Analysis):
   - Για να βελτιώσουμε την απόδοση του συνολικού καλύτερου μοντέλου, χρησιμοποιήσαμε τη μέθοδο **PCA**, διατηρώντας 139 χαρακτηριστικά (principal   components).
   - Ο αριθμός αυτός επιλέχθηκε μετά από διαδικασία tuning στο εύρος [1, 224].
   - Εφαρμόσαμε τη μέθοδο PCA στο 80% του training set και εκπαιδεύσαμε ξανά το μοντέλο. Ως αποτέλεσμα, η ακρίβεια του μοντέλου στο validation set αυξήθηκε.

### Επανεκπαίδευση με PCA στο συνολικό training set:
   - Για να εκμεταλλευτούμε τη βελτίωση αυτή, αποφασίσαμε να επανεκπαιδεύσουμε το καλύτερο μοντέλο με την τεχνική PCA στο συνολικό training set.

### Προβλέψεις του τελικού εκπαιδευμένου μοντέλου στο test set:
   - Το τελικό εκπαιδευμένο μοντέλο πραγματοποιεί προβλέψεις στο **test set**.
   - Έτσι, παράγεται το αρχείο **labels25.npy**.

---

### Φόρτωση των δεδομένων και χωρισμός σε 80% training set και 20% validation set

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from itertools import product
from sklearn.decomposition import PCA


# Load the Training and Test Data
train_data = pd.read_csv('datasetTV.csv', header=None)  # Training data
test_data = pd.read_csv('datasetTest.csv', header=None)  # Test data

# Preprocess the Data
# Separate features (X) and target labels (y) in the training data
X_train_full = train_data.iloc[:, :-1].values  # All columns except the last column (features)
y_train_full = train_data.iloc[:, -1].values  # Last column (target labels)
X_test = test_data.values  # Test data features (all columns)

# Split the training data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

---

### Ακολουθεί η συνάρτηση που δημιουργήσαμε για να κάνουμε manual hyperparameter tuning για κάθε αλγόριθμο ταξινόμησης

In [2]:
def manual_hyperparameter_tuning(algorithms, param_grids, X_train, y_train, X_val, y_val):
    """
    Perform hyperparameter tuning for multiple machine learning models.

    Parameters:
    algorithms (dict): A dictionary of model names and their corresponding classifiers.
    param_grids (dict): A dictionary containing hyperparameter grids for each model.
    X_train (array): Training features.
    y_train (array): Training labels.
    X_val (array): Validation features.
    y_val (array): Validation labels.

    Returns:
    dict: Best models, best accuracies, and best parameters for each model.
    """
    # Dictionaries to store the best model, best accuracy, and best hyperparameters for each model.
    best_model_for_all = {}
    best_accuracy_for_all = {}
    best_params_for_all = {}

    # Perform hyperparameter tuning for each model using the defined grids
    for model_name, model in algorithms.items():
        print(f"Training and evaluating {model_name}...")

        # Get the corresponding parameter grid
        param_grid = param_grids[model_name]

        # Generate all possible combinations of hyperparameters
        param_combinations = list(product(*param_grid.values()))

        best_accuracy = 0  # Initialize best accuracy for comparison
        best_params = None  # Initialize best parameters for comparison

        # Iterate over each hyperparameter combination
        for params in param_combinations:
            param_dict = dict(zip(param_grid.keys(), params))  # Create a dictionary of hyperparameters
            model.set_params(**param_dict)  # Set the model's hyperparameters
            model.fit(X_train, y_train)  # Train the model on the training set
            y_val_pred = model.predict(X_val)  # Make predictions on the validation set
            val_accuracy = accuracy_score(y_val, y_val_pred)  # Calculate the accuracy on validation set

            # Update the best accuracy and parameters if the current model is better
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_model = model
                best_params = param_dict

        # Store the results for this model
        best_model_for_all[model_name] = best_model
        best_accuracy_for_all[model_name] = best_accuracy
        best_params_for_all[model_name] = best_params

        # Print results for this model
        print(f"Best model for {model_name} with accuracy: {best_accuracy * 100:.2f}% and parameters: {best_params}\n")

    return best_model_for_all, best_accuracy_for_all, best_params_for_all

---

### **Όλοι οι συνδυασμοί υπερπαραμέτρων** που δοκιμάστηκαν με την manual_hyperparameter_tuning για τους αλγόριθμους ταξινόμησης είναι οι εξής:

In [3]:
# Define the hyperparameter grids for each model


# All the hyperparameters combinations for Random Forest
param_grid_rf_all = {
    'n_estimators': [200, 300, 400],
    'max_depth': [10, 20, 30, 40, 50],
    'max_samples': [0.7, 1.0]
}

# All the hyperparameters combinations for SVM
param_grid_svm_all = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'gamma': ['scale', 'auto'],
    'kernel': ['linear', 'rbf', 'poly'],
    'tol': [1e-9, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
    'coef0': [0.001, 0.01, 0.1, 0.2, 0.5, 1.0],
    'class_weight': [None, 'balanced']

}

# All the hyperparameters combinations for Naive Bayes
param_grid_nb_all = {
    'var_smoothing': [1e-9, 1e-8, 1e-7]
}

# All the hyperparameters combinations for MLP
param_grid_mlp_all = {
    'hidden_layer_sizes': [(50,), (100,), (200,), (300,), (400,), (500,), (600,), (700,),
                           (100, 100), (200, 200), (300, 300), (400, 400), (500, 500), (600, 600),
                           (700, 700), (800, 800), (900, 900),
                           (1000, 1000), (100, 100, 50), (100, 100, 100), (200, 200, 200), (300, 300, 200),
                           (300, 300, 300), (400, 400, 400), (500, 500, 500), (600, 600, 600),
                           (700, 700, 700), (800, 800, 800), (900, 900, 900), (1000, 1000, 1000)],
    'activation': ['relu', 'logistic', 'tanh'],
    'solver': ['adam', 'lbfgs', 'sgd'],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [2000],
    'alpha': [0.00001, 0.0001, 0.001],
    'epsilon': [1e-6, 1e-7, 1e-8, 1e-9, 1e-10]
}

# All the hyperparameters combinations for KNN
param_grid_knn_all = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# All the hyperparameters combinations for Decision Tree
param_grid_dt_all = {
    'max_depth': range(1, 50)
}

# All the hyperparameters combinations for Logistic Regression
param_grid_lr_all = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear', 'saga', 'newton-cg'],
    'max_iter': [1000, 2000],
    'class_weight': [None, 'balanced'],
    'tol': [1e-4, 1e-3, 1e-2],
}

---

### Τα **καλύτερα μοντέλα για κάθε αλγόριθμο** προέκυψαν μετά από **αρκετές κλήσεις** της συνάρτησης manual_hyperparameter_tuning. 
### Οι κλήσεις αφορούσαν όλους τους παραπάνω συνδυασμούς υπερπαραμέτρων και τα αποτελέσματα είναι τα εξής:

In [4]:
# The hyperparameters of the best model for Random Forest
param_grid_rf = {
    'n_estimators': [300],  # Number of trees in the forest
    'max_depth': [40],  # Maximum depth of trees
    'max_samples': [1.0]   # Proportion of the dataset used to train each tree
}

# The hyperparameters of the best model for SVM
param_grid_svm = {
    'C': [1],  # Regularization strength
    'gamma': ['scale'],  # Kernel coefficient
    'kernel': ['poly'],  # Polynomial kernel
    'tol': [1e-3],   # Tolerance for stopping criteria
    'coef0': [0.2],  # Independent term in the kernel function
    'class_weight': ['balanced']  # Adjust weights to handle class imbalance
}

# The hyperparameters of the best model for Naive Bayes
param_grid_nb = {
    'var_smoothing': [1e-9]   # Smoothing parameter to avoid zero variance
}

# The hyperparameters of the best model for MLP
param_grid_mlp = {
    'hidden_layer_sizes': [(300,)],  # One hidden layer with 300 neurons
    'activation': ['relu'],  # Activation function
    'solver': ['adam'],  # Solver for weight optimization
    'learning_rate': ['constant'],  # Constant learning rate
    'max_iter': [2000],  # Maximum number of iterations
    'alpha': [0.0001],   # Regularization term
    'epsilon': [1e-7]  # Tolerance for optimization
}

# The hyperparameters of the best model for KNN
param_grid_knn = {
    'n_neighbors': [5],  # Number of neighbors to use in classification
    'weights': ['distance'],  # Use distance weighting for neighbors
    'metric': ['euclidean']  # Use Euclidean distance metric
}

# The hyperparameters of the best model for Decision Tree
param_grid_dt = {
    'max_depth': [11]  # Maximum depth of the decision tree
}

# The hyperparameters of the best model for Logistic Regression
param_grid_lr = {
    'C': [0.1],  # Regularization strength for logistic regression
    'solver': ['lbfgs'],   # Solver for optimization
    'max_iter': [1000],  # Maximum iterations for the solver
    'class_weight': [None],  # No adjustment for class imbalance
    'tol': [1e-2],  # Tolerance for optimization
}

---

### Στα πλαίσια του Μέρους Δ, καλούμε τη συνάρτηση manual_hyperparameter_tuning για να υπολογίσουμε την ακρίβεια στο validation set **ΜΟΝΟ** για το καλύτερο μοντέλο κάθε αλγορίθμου. 

### Η κλήση της συνάρτησης **για όλους τους δυνατούς συνδυασμούς υπερπαραμέτρων** σε κάθε αλγόριθμο, θα απαιτούσε υπερβολικά μεγάλο χρόνο εκτέλεσης.

In [5]:
# Define the machine learning algorithms
algorithms = {
    "Random Forest": RandomForestClassifier(random_state=42),  # Random Forest algorithm
    "SVM": SVC(random_state=42),  # Support Vector Machine (SVM) algorithm
    "Naive Bayes": GaussianNB(),  # Gaussian Naive Bayes algorithm
    "MLP": MLPClassifier(random_state=42),  # Multilayer Perceptron (MLP) algorithm
    "KNN": KNeighborsClassifier(),  # k-Nearest Neighbors (KNN) algorithm
    "Decision Tree": DecisionTreeClassifier(random_state=42),  # Decision Tree algorithm
    "Logistic Regression": LogisticRegression(random_state=42)  # Multinomial Logistic Regression algorithm
}

# Define the parameter grids ONLY for the BEST MODEL of each algorithm.
param_grids = {
    "Random Forest": param_grid_rf,  # Parameter grid for Random Forest
    "SVM": param_grid_svm,  # Parameter grid for SVM
    "Naive Bayes": param_grid_nb,  # Parameter grid for Gaussian Naive Bayes
    "MLP": param_grid_mlp,  # Parameter grid for MLP
    "KNN": param_grid_knn,  # Parameter grid for KNN
    "Decision Tree": param_grid_dt,  # Parameter grid for Decision Tree
    "Logistic Regression": param_grid_lr  # Parameter grid for Multinomial Logistic Regression
}

# Call the manual_hyperparameter_tuning function.
# It returns the best model, best accuracy, and best parameters for each algorithm.
best_model_for_all, best_accuracy_for_all, best_params_for_all = manual_hyperparameter_tuning(algorithms,
                                                                                              param_grids,
                                                                                              X_train,
                                                                                              y_train,
                                                                                              X_val,
                                                                                              y_val)

Training and evaluating Random Forest...
Best model for Random Forest with accuracy: 81.93% and parameters: {'n_estimators': 300, 'max_depth': 40, 'max_samples': 1.0}

Training and evaluating SVM...
Best model for SVM with accuracy: 87.36% and parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'poly', 'tol': 0.001, 'coef0': 0.2, 'class_weight': 'balanced'}

Training and evaluating Naive Bayes...
Best model for Naive Bayes with accuracy: 69.87% and parameters: {'var_smoothing': 1e-09}

Training and evaluating MLP...
Best model for MLP with accuracy: 86.34% and parameters: {'hidden_layer_sizes': (300,), 'activation': 'relu', 'solver': 'adam', 'learning_rate': 'constant', 'max_iter': 2000, 'alpha': 0.0001, 'epsilon': 1e-07}

Training and evaluating KNN...
Best model for KNN with accuracy: 84.96% and parameters: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'euclidean'}

Training and evaluating Decision Tree...
Best model for Decision Tree with accuracy: 65.24% and parameters: {'max_de

---

### **Επιλογή του συνολικού καλύτερου μοντέλου**: Επιλέγεται το μοντέλο με την καλύτερη ακρίβεια στο validation set.

In [6]:
# Select the final best model with the highest accuracy in validation set.
best_model_name = max(best_accuracy_for_all, key=best_accuracy_for_all.get)
best_model = best_model_for_all[best_model_name]
best_params = best_params_for_all[best_model_name]

print(f"The Final Best Model is {best_model_name} with best parameters: {best_params}\n")

The Final Best Model is SVM with best parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'poly', 'tol': 0.001, 'coef0': 0.2, 'class_weight': 'balanced'}



---

### **Βελτίωση απόδοσης με PCA (Principal Component Analysis)**: Εφαρμόσαμε τη μέθοδο PCA στο 80% του training set και εκπαιδεύσαμε ξανά το συνολικό καλύτερο μοντέλο. 
### **Αποτέλεσμα**: Η ακρίβεια του μοντέλου στο validation set αυξήθηκε.

In [7]:
# Apply PCA (Principal Component Analysis) to reduce the dimensionality of the data
pca = PCA(n_components=139)  # Use 139 principal components
print(pca)
X_pca = pca.fit_transform(X_train_full)  # Apply PCA on the entire dataset

# Split the PCA-transformed data into training and validation sets (80% train set and 20% validation set)
X_train_pca, X_val_pca, y_train_pca, y_val_pca = train_test_split(X_pca, y_train_full, test_size=0.2, random_state=42)

# Train the best model on the 80% of the total training set with PCA applied
print(f"Training the best model ({best_model_name}) on the training set with PCA...\n")

best_model.set_params(**best_params)
best_model.fit(X_train_pca, y_train_pca)  # Train the model on the PCA-transformed training set

# Evaluate the model on the validation set
val_predictions = best_model.predict(X_val_pca)  # Make predictions on the validation set
val_accuracy = accuracy_score(y_val_pca, val_predictions)  # Calculate the validation accuracy

best_model_accuracy_before_pca = best_accuracy_for_all[best_model_name]
improvement = (val_accuracy * 100) - (best_model_accuracy_before_pca * 100)
print(f"Validation Accuracy for {best_model_name} with PCA: {val_accuracy * 100:.2f}%\n")
print(f"Improvement of accuracy with PCA : {val_accuracy * 100:.2f} - {best_model_accuracy_before_pca * 100:.2f} = "
      f"{improvement:.2f}%\n")

# We notice that the PCA trains our best model in a better way for this specific training set.
# Because the validation accuracy is better with PCA.
print(f"So the PCA method trains better the best model\n")

PCA(n_components=139)
Training the best model (SVM) on the training set with PCA...

Validation Accuracy for SVM with PCA: 88.56%

Improvement of accuracy with PCA : 88.56 - 87.36 = 1.20%

So the PCA method trains better the best model



---

### Επανεκπαίδευση του καλύτερου μοντέλου με PCA στο **συνολικό training set**.
### Το τελικό εκπαιδευμένο μοντέλο πραγματοποιεί προβλέψεις στο test set.

In [8]:
# Re-train the best model using the entire training set (with PCA applied)
print(f"Re-training the best model ({best_model_name}) on the entire training set with PCA...")

# Apply PCA on the entire dataset again (including the validation set, as it's already been split)
X_train_full_pca = pca.transform(X_train_full)  # Transform the entire dataset with the same PCA transformation
best_model.fit(X_train_full_pca, y_train_full)  # Train the model on the full PCA-transformed training set

# Make predictions on the test set (with PCA applied)
X_test_pca = pca.transform(X_test)  # Apply PCA transformation to the test set
test_predictions = best_model.predict(X_test_pca)  # Predict on the PCA-transformed test set

print(f"Test Predictions using {best_model_name}: {test_predictions}")
print(f"Number of labels predicted: {len(test_predictions)}")

Re-training the best model (SVM) on the entire training set with PCA...
Test Predictions using SVM: [1 4 5 ... 3 5 1]
Number of labels predicted: 6955


---

### Αποθήκευση των προβλέψεων στο labels25.npy και έλεγχος της εγκυρότητας των αποτελεσμάτων.

In [9]:
# test_predictions is the array with our best model predictions on test set.
if len(test_predictions) == 6955:  # Check if the number of predictions is correct
    # Save the predictions as a NumPy file
    np.save("labels25.npy", np.array(test_predictions))
    print(f"Predictions have been saved to 'labels25.npy'.")
else:
    print(f"Error: The number of test predictions is {len(test_predictions)}, but it should be 6955.")

# Check the .npy file
# Load the labels from the .npy file
try:
    labels = np.load('labels25.npy')  # Load the saved predictions from the .npy file
    print(f"Labels loaded successfully! Number of labels: {len(labels)}")
    print(f"Shape of the labels array: {labels.shape}")  # Print the shape of the labels array

    # Check if the labels are between 1 and 5 and each label appears at least once
    unique_labels = np.unique(labels)  # Get unique labels from the predictions

    # Check if the unique labels are between 1 and 5
    if np.all(np.isin(unique_labels, [1, 2, 3, 4, 5])):
        print("The labels are from 1 to 5.")
    else:
        print(f"Error: The labels contain values outside of 1 to 5: {unique_labels}")

    # Check if each label appears at least once
    label_counts = {label: np.sum(labels == label) for label in [1, 2, 3, 4, 5]}
    missing_labels = [label for label, count in label_counts.items() if count == 0]

    if not missing_labels:
        print("All labels (1 to 5) appear at least once.")
    else:
        print(f"Error: The following labels are missing in the predictions: {missing_labels}")

except FileNotFoundError:
    print("Error: The file 'labels25.npy' could not be found.")
except Exception as e:
    print(f"An error occurred: {e}")

Predictions have been saved to 'labels25.npy'.
Labels loaded successfully! Number of labels: 6955
Shape of the labels array: (6955,)
The labels are from 1 to 5.
All labels (1 to 5) appear at least once.


### Τέλος, μια γενική παρατήρηση για την εκπαίδευση των μοντέλων στο Μέρος Δ:
#### Χρησιμοποιούμε ένα σταθερό 20% του συνόλου των 8743 δειγμάτων για την αξιολόγηση (validation) σε κάθε εκτέλεση, με στόχο την εξοικονόμηση χρόνου. Έχουμε διαπιστώσει ότι τα αποτελέσματα που προκύπτουν με αυτήν τη μέθοδο είναι συγκρίσιμα με εκείνα που προκύπτουν από το cross-validation, το οποίο εξετάζει διαφορετικά υποσύνολα του dataset. Επομένως, θεωρούμε ότι το σταθερό 20% είναι επαρκές για την αξιόπιστη αξιολόγηση των μοντέλων μας. Αυτή η προσέγγιση διασφαλίζει τη συνέπεια στη σύγκριση διαφορετικών μοντέλων, ενώ παράλληλα μειώνει τον κίνδυνο overfitting ή underfitting στο τελικό test set. 